# HW: X-ray images classification
--------------------------------------

Before you begin, open Mobaxterm and connect to triton with the user and password you were give with. Activate the environment `2ndPaper` and then type the command `pip install scikit-image`.

In this assignment you will be dealing with classification of 32X32 X-ray images of the chest. The image can be classified into one of four options: lungs (l), clavicles (c), and heart (h) and background (b). Even though those labels are dependent, we will treat this task as multiclass and not as multilabel. The dataset for this assignment is located on a shared folder on triton (`/MLdata/MLcourse/X_ray/'`).

In [1]:
import os
import numpy as np
from tensorflow.keras.layers import Dense, MaxPool2D, Conv2D, Dropout
from tensorflow.keras.layers import Flatten, InputLayer
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import *

from tensorflow.keras.initializers import Constant
from tensorflow.keras.datasets import fashion_mnist
import tensorflow.keras.backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from skimage.io import imread

from skimage.transform import rescale, resize, downscale_local_mean
# %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options =
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

In [3]:
def preprocess(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        src = imread(os.path.join(datapath, fn),1)
        img = resize(src,(32,32),order = 3)
        
        images[ii,:,:,0] = img
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    BaseImages = images
    BaseY = Y
    return BaseImages, BaseY

In [4]:
def preprocess_train_and_val(datapath):
    # This part reads the images
    classes = ['b','c','l','h']
    imagelist = [fn for fn in os.listdir(datapath)]
    N = len(imagelist)
    num_classes = len(classes)
    images = np.zeros((N, 32, 32, 1))
    Y = np.zeros((N,num_classes))
    ii=0
    for fn in imagelist:

        images[ii,:,:,0] = imread(os.path.join(datapath, fn),1)
        cc = -1
        for cl in range(len(classes)):
            if fn[-5] == classes[cl]:
                cc = cl
        Y[ii,cc]=1
        ii += 1

    return images, Y

In [5]:
#Loading the data for training and validation:
src_data = 'X_ray/' #/MLdata/MLcourse/
train_path = src_data + 'train'
val_path = src_data + 'validation'
test_path = src_data + 'test'
BaseX_train , BaseY_train = preprocess_train_and_val(train_path)
BaseX_val , BaseY_val = preprocess_train_and_val(val_path)
X_test, Y_test = preprocess(test_path)

In [6]:
keras.backend.clear_session()

### PART 1: Fully connected layers 
--------------------------------------

---
<span style="color:red">***Task 1:***</span> *NN with fully connected layers. 

Elaborate a NN with 2 hidden fully connected layers with 300, 150 neurons and 4 neurons for classification. Use ReLU activation functions for the hidden layers and He_normal for initialization. Don't forget to flatten your image before feedforward to the first dense layer. Name the model `model_relu`.*

---

In [7]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
model_relu = Sequential()
model_relu.add(Flatten(input_shape=(32,32,)))
model_relu.add(Dense(n_hidden_start,kernel_initializer="HeNormal",activation='relu'))
model_relu.add(Dense(int(n_hidden_start/2),kernel_initializer="HeNormal", activation='relu'))
model_relu.add(Dense(int(n_hidden_start/75),kernel_initializer="HeNormal", activation='sigmoid'))

#----------------------------------------------------------------------------------------

In [8]:
model_relu.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 300)               307500    
_________________________________________________________________
dense_1 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


Compile the model with the optimizer above, accuracy metric and adequate loss for multiclass task. Train your model on the training set and evaluate the model on the testing set. Print the accuracy and loss over the testing set.

In [10]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')

if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_relu"
model_path = os.path.join(save_dir, model_name)
model_relu.save(model_path)

history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,validation_data=(BaseX_val,BaseY_val))

y_pred_test = model_relu.predict(X_test)
print("Evaluate on test data")
results = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)

#----------------------------------------------------------------------------------------

INFO:tensorflow:Assets written to: results/init_weigths_relu\assets
Epoch 1/25
102/102 [==============================] - 2s 13ms/step - loss: 1.3691 - accuracy: 0.3108 - val_loss: 1.1685 - val_accuracy: 0.5584
Epoch 2/25
102/102 [==============================] - 0s 2ms/step - loss: 1.1392 - accuracy: 0.5935 - val_loss: 1.0448 - val_accuracy: 0.6499
Epoch 3/25
102/102 [==============================] - 0s 2ms/step - loss: 1.0370 - accuracy: 0.6620 - val_loss: 0.9682 - val_accuracy: 0.6840
Epoch 4/25
102/102 [==============================] - 0s 2ms/step - loss: 0.9658 - accuracy: 0.7067 - val_loss: 0.9167 - val_accuracy: 0.7199
Epoch 5/25
102/102 [==============================] - 0s 2ms/step - loss: 0.9086 - accuracy: 0.7216 - val_loss: 0.8743 - val_accuracy: 0.7384
Epoch 6/25
102/102 [==============================] - 0s 2ms/step - loss: 0.8642 - accuracy: 0.7481 - val_loss: 0.8394 - val_accuracy: 0.7448
Epoch 7/25
102/102 [==============================] - 0s 2ms/step - loss: 0.834

---
<span style="color:red">***Task 2:***</span> *Activation functions.* 

Change the activation functions to LeakyRelu or tanh or sigmoid. Name the new model `new_a_model`. Explain how it can affect the model.*

---

In [11]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=(32,32,)))
new_a_model.add(Dense(n_hidden_start,kernel_initializer="HeNormal",activation='sigmoid'))
new_a_model.add(Dense(int(n_hidden_start/2),kernel_initializer="HeNormal", activation='sigmoid'))
new_a_model.add(Dense(int(n_hidden_start/75),kernel_initializer="HeNormal", activation='sigmoid'))

#----------------------------------------------------------------------------------------

In [12]:
new_a_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 300)               307500    
_________________________________________________________________
dense_4 (Dense)              (None, 150)               45150     
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 604       
Total params: 353,254
Trainable params: 353,254
Non-trainable params: 0
_________________________________________________________________


---
<span style="color:red">***Task 3:***</span> *Number of epochs.* 

Train the new model using 25 and 40 epochs. What difference does it makes in term of performance? Remember to save the compiled model for having initialized weights for every run as we did in tutorial 12. Evaluate each trained model on the test set*

---

In [13]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 25

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [14]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')
# Code taken from TUTORIAL 11:::::
if not("results" in os.listdir()):
    os.mkdir("results")
save_dir = "results/"
model_name = "init_weigths_newmodel"
model_path = os.path.join(save_dir, model_name)
new_a_model.save(model_path)
# print('Saved initialized model at %s ' % model_path)

history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,validation_data = (BaseX_val,BaseY_val))

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)

#-----------------------------------------------------------------------------------------

INFO:tensorflow:Assets written to: results/init_weigths_newmodel\assets
Epoch 1/25
102/102 [==============================] - 1s 4ms/step - loss: 1.5316 - accuracy: 0.2412 - val_loss: 1.4317 - val_accuracy: 0.2500
Epoch 2/25
102/102 [==============================] - 0s 2ms/step - loss: 1.4146 - accuracy: 0.2596 - val_loss: 1.3827 - val_accuracy: 0.3501
Epoch 3/25
102/102 [==============================] - 0s 2ms/step - loss: 1.3743 - accuracy: 0.3717 - val_loss: 1.3566 - val_accuracy: 0.4381
Epoch 4/25
102/102 [==============================] - 0s 2ms/step - loss: 1.3516 - accuracy: 0.4544 - val_loss: 1.3324 - val_accuracy: 0.4826
Epoch 5/25
102/102 [==============================] - 0s 2ms/step - loss: 1.3277 - accuracy: 0.4853 - val_loss: 1.3090 - val_accuracy: 0.5041
Epoch 6/25
102/102 [==============================] - 0s 2ms/step - loss: 1.3048 - accuracy: 0.4998 - val_loss: 1.2862 - val_accuracy: 0.5156
Epoch 7/25
102/102 [==============================] - 0s 2ms/step - loss: 1.

In [15]:
#Inputs: 
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 0
batch_size = 64
epochs = 40

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)



In [16]:
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')

new_a_model = tf.keras.models.load_model("results/init_weigths_newmodel") # Initializing weights before total run

history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,validation_data=(BaseX_val,BaseY_val))

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#-----------------------------------------------------------------------------------------

Epoch 1/40
102/102 [==============================] - 1s 4ms/step - loss: 1.4983 - accuracy: 0.2447 - val_loss: 1.4335 - val_accuracy: 0.2315
Epoch 2/40
102/102 [==============================] - 0s 3ms/step - loss: 1.4053 - accuracy: 0.2831 - val_loss: 1.3829 - val_accuracy: 0.3484
Epoch 3/40
102/102 [==============================] - 0s 2ms/step - loss: 1.3698 - accuracy: 0.4032 - val_loss: 1.3562 - val_accuracy: 0.4427
Epoch 4/40
102/102 [==============================] - 0s 2ms/step - loss: 1.3448 - accuracy: 0.4699 - val_loss: 1.3317 - val_accuracy: 0.4745
Epoch 5/40
102/102 [==============================] - 0s 2ms/step - loss: 1.3216 - accuracy: 0.4898 - val_loss: 1.3084 - val_accuracy: 0.5012
Epoch 6/40
102/102 [==============================] - 0s 2ms/step - loss: 1.2990 - accuracy: 0.5139 - val_loss: 1.2857 - val_accuracy: 0.5255
Epoch 7/40
102/102 [==============================] - 0s 2ms/step - loss: 1.2771 - accuracy: 0.5270 - val_loss: 1.2633 - val_accuracy: 0.5376
Epoch 

---
<span style="color:red">***Task 4:***</span> *Mini-batches.* 

Build the `model_relu` again and run it with a batch size of 32 instead of 64. What are the advantages of the mini-batch vs. SGD?*

---

In [17]:
keras.backend.clear_session()

In [18]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
model_relu = Sequential()
model_relu.add(Flatten(input_shape=(32,32,)))
model_relu.add(Dense(n_hidden_start,kernel_initializer="HeNormal", activation='relu'))
model_relu.add(Dense(int(n_hidden_start/2),kernel_initializer="HeNormal", activation='relu'))
model_relu.add(Dense(int(n_hidden_start/75),kernel_initializer="HeNormal", activation='sigmoid'))
#----------------------------------------------------------------------------------------

In [19]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)


In [20]:
#--------------------------Impelment your code here:-------------------------------------
model_relu.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')

model_relu = tf.keras.models.load_model("results/init_weigths_relu")
history_relu = model_relu.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,validation_data=(BaseX_val,BaseY_val))

y_pred_test = model_relu.predict(X_test)
print("Evaluate on test data")
results = model_relu.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 1s 2ms/step - loss: 1.2326 - accuracy: 0.4765 - val_loss: 1.0844 - val_accuracy: 0.6262
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 1.0253 - accuracy: 0.6645 - val_loss: 0.9594 - val_accuracy: 0.6973
Epoch 3/50
203/203 [==============================] - 0s 2ms/step - loss: 0.9248 - accuracy: 0.7169 - val_loss: 0.8847 - val_accuracy: 0.7251
Epoch 4/50
203/203 [==============================] - 0s 2ms/step - loss: 0.8584 - accuracy: 0.7487 - val_loss: 0.8307 - val_accuracy: 0.7431
Epoch 5/50
203/203 [==============================] - 0s 2ms/step - loss: 0.8085 - accuracy: 0.7600 - val_loss: 0.7906 - val_accuracy: 0.7587
Epoch 6/50
203/203 [==============================] - 0s 2ms/step - loss: 0.7690 - accuracy: 0.7740 - val_loss: 0.7588 - val_accuracy: 0.7662
Epoch 7/50
203/203 [==============================] - 0s 2ms/step - loss: 0.7345 - accuracy: 0.7847 - val_loss: 0.7264 - val_accuracy: 0.7743
Epoch 

---
<span style="color:red">***Task 4:***</span> *Batch normalization.* 

Build the `new_a_model` again and add batch normalization layers. How does it impact your results?*

---

In [21]:
keras.backend.clear_session()

In [22]:
#--------------------------Impelment your code here:-------------------------------------
n_hidden_start=300
new_a_model = Sequential()
new_a_model.add(Flatten(input_shape=(32,32,)))
new_a_model.add(Dense(n_hidden_start,kernel_initializer="HeNormal",activation='sigmoid'))
new_a_model.add(BatchNormalization())
new_a_model.add(Dense(int(n_hidden_start/2), kernel_initializer="HeNormal",activation='sigmoid'))
new_a_model.add(BatchNormalization())
new_a_model.add(Dense(int(n_hidden_start/75), kernel_initializer="HeNormal",activation='sigmoid'))
new_a_model.add(BatchNormalization())
#---------------------------------------------------------------------------------------

In [23]:
batch_size = 32
epochs = 50

#Define your optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)
#Compile the network: 


In [24]:
#Preforming the training by using fit 
#--------------------------Impelment your code here:-------------------------------------
new_a_model.compile(optimizer=AdamOpt,metrics=['accuracy'], loss='categorical_crossentropy')
new_a_model = tf.keras.models.load_model("results/init_weigths_newmodel") 
history_model_a = new_a_model.fit(BaseX_train, BaseY_train, batch_size=batch_size, epochs=epochs,validation_data = (BaseX_val,BaseY_val))

y_pred_test = new_a_model.predict(X_test)
print("Evaluate on test data")
results = new_a_model.evaluate(X_test, Y_test, batch_size=batch_size)
print("test loss, test acc:", results)
#----------------------------------------------------------------------------------------

Epoch 1/50
203/203 [==============================] - 1s 3ms/step - loss: 1.4578 - accuracy: 0.2575 - val_loss: 1.3916 - val_accuracy: 0.3438
Epoch 2/50
203/203 [==============================] - 0s 2ms/step - loss: 1.3694 - accuracy: 0.3951 - val_loss: 1.3474 - val_accuracy: 0.4630
Epoch 3/50
203/203 [==============================] - 0s 2ms/step - loss: 1.3295 - accuracy: 0.4869 - val_loss: 1.3093 - val_accuracy: 0.4913
Epoch 4/50
203/203 [==============================] - 0s 2ms/step - loss: 1.2926 - accuracy: 0.5153 - val_loss: 1.2727 - val_accuracy: 0.5214
Epoch 5/50
203/203 [==============================] - 0s 2ms/step - loss: 1.2577 - accuracy: 0.5411 - val_loss: 1.2371 - val_accuracy: 0.5411
Epoch 6/50
203/203 [==============================] - 0s 2ms/step - loss: 1.2245 - accuracy: 0.5598 - val_loss: 1.2047 - val_accuracy: 0.5677
Epoch 7/50
203/203 [==============================] - 0s 2ms/step - loss: 1.1941 - accuracy: 0.5690 - val_loss: 1.1758 - val_accuracy: 0.5856
Epoch 

### PART 2: Convolutional Neural Network (CNN)
------------------------------------------------------------------------------------

---
<span style="color:red">***Task 1:***</span> *2D CNN.* 

Have a look at the model below and answer the following:
* How many layers does it have?
* How many filter in each layer?
* Would the number of parmaters be similar to a fully connected NN?
* Is this specific NN performing regularization?

---

In [25]:
def get_net(input_shape,drop,dropRate,reg):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

In [26]:
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet = get_net(input_shape,drop,dropRate,reg)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute (Permute)            (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 128)      

In [27]:
NNet=get_net(input_shape,drop,dropRate,reg)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_1 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 64)        640       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 32, 32, 64)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
dropout_7 (Dropout)          (None, 16, 16, 128)      

In [28]:
from tensorflow.keras.optimizers import *
import os
from tensorflow.keras.callbacks import *

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')

#Saving checkpoints during training:
Checkpath = os.getcwd()
Checkp = ModelCheckpoint(Checkpath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, save_freq=1)

In [29]:
#Preforming the training by using fit 
# IMPORTANT NOTE: This will take a few minutes!
h = NNet.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
#NNet.save(model_fn)

Epoch 1/25
102/102 [==============================] - 51s 492ms/step - loss: 8.2400 - acc: 0.3780 - val_loss: 8.0294 - val_acc: 0.2500
Epoch 2/25
102/102 [==============================] - 49s 476ms/step - loss: 7.6110 - acc: 0.5273 - val_loss: 8.1661 - val_acc: 0.2500
Epoch 3/25
102/102 [==============================] - 49s 484ms/step - loss: 7.4201 - acc: 0.5844 - val_loss: 8.2606 - val_acc: 0.2575
Epoch 4/25
102/102 [==============================] - 49s 479ms/step - loss: 7.3105 - acc: 0.6202 - val_loss: 8.2584 - val_acc: 0.2928
Epoch 5/25
102/102 [==============================] - 50s 486ms/step - loss: 7.2100 - acc: 0.6479 - val_loss: 8.1471 - val_acc: 0.2784
Epoch 6/25
102/102 [==============================] - 49s 479ms/step - loss: 7.1166 - acc: 0.6695 - val_loss: 7.9632 - val_acc: 0.2847
Epoch 7/25
102/102 [==============================] - 49s 478ms/step - loss: 7.0929 - acc: 0.6743 - val_loss: 7.7808 - val_acc: 0.3252
Epoch 8/25
102/102 [==============================] - 4

In [30]:
# NNet.load_weights('Weights_1.h5')

In [31]:
results = NNet.evaluate(X_test,Y_test)
print('test loss, test acc:', results)

6/6 [==============================] - 0s 39ms/step - loss: 7.7583 - acc: 0.3543
test loss, test acc: [7.758265972137451, 0.35428571701049805]


---
<span style="color:red">***Task 2:***</span> *Number of filters* 

Rebuild the function `get_net` to have as an input argument a list of number of filters in each layers, i.e. for the CNN defined above the input should have been `[64, 128, 128, 256, 256]`. Now train the model with the number of filters reduced by half. What were the results.

---

In [32]:
#--------------------------Impelment your code here:-------------------------------------
def get_net1(input_shape,drop,dropRate,reg,num_filters):
    #Defining the network architecture:
    model = Sequential()
    model.add(Permute((1,2,3),input_shape = input_shape))
    model.add(Conv2D(filters=num_filters[0], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_1',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=num_filters[1], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_2',kernel_regularizer=regularizers.l2(reg)))
    if drop:    
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=num_filters[2], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_3',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=num_filters[3], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_4',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(Conv2D(filters=num_filters[4], kernel_size=(3,3), padding='same', activation='relu',name='Conv2D_5',kernel_regularizer=regularizers.l2(reg)))
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(BatchNormalization(axis=1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    #Fully connected network tail:      
    model.add(Dense(512, activation='elu',name='FCN_1')) 
    if drop:
        model.add(Dropout(rate=dropRate))
    model.add(Dense(128, activation='elu',name='FCN_2'))
    model.add(Dense(4, activation= 'softmax',name='FCN_3'))
    model.summary()
    return model

num_filters = [32, 64, 64, 128, 128]
input_shape = (32,32,1)
learn_rate = 1e-5
decay = 1e-03
batch_size = 64
epochs = 25
drop = True
dropRate = 0.3
reg = 1e-2
NNet_new = get_net1(input_shape,drop,dropRate,reg,num_filters)

#Defining the optimizar parameters:
AdamOpt = Adam(lr=learn_rate,decay=decay)

#Compile the network: 
NNet_new.compile(optimizer=AdamOpt, metrics=['acc'], loss='categorical_crossentropy')


h = NNet_new.fit(x=BaseX_train, y=BaseY_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0, validation_data = (BaseX_val, BaseY_val), shuffle=True)
results = NNet_new.evaluate(X_test,Y_test)
print('test loss, test acc:', results)
#----------------------------------------------------------------------------------------

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
permute_2 (Permute)          (None, 32, 32, 1)         0         
_________________________________________________________________
Conv2D_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
dropout_12 (Dropout)         (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
Conv2D_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_13 (Dropout)         (None, 16, 16, 64)       

That's all folks! See you :)